Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [3]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [5]:
#print out data sample

In [6]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [7]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [ ]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

In [8]:
grid_search.best_score_

NameError: name 'grid_search' is not defined

In [9]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

NameError: name 'grid_search' is not defined

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [10]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [11]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [12]:
from xgboost import XGBClassifier

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = XGBClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [13]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'vect__max_df': [x/100. for x in range(75, 101)],
    'vect__min_df': [x/100. for x in range(2, 6)],
    'clf__max_depth': range(5, 26, 5),
    'clf__learning_rate': [x/100. for x in range(5, 36)],
    'clf__min_child_weight': range(1, 36, 5),
}

grid_search = RandomizedSearchCV(pipe, parameters, cv=4, n_jobs=-1, verbose=1, n_iter=250)
grid_search.fit(train["description"], train["ratingCategory"])

Fitting 4 folds for each of 250 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 10.4min


KeyboardInterrupt: 

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

You're trying to achieve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [18]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [27]:
params = { 
    'lsi__svd__n_components': [80, 100, 120],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators': range(80, 141, 20),
    'clf__max_depth': range(5, 26, 5),
    'clf__learning_rate': [x/100. for x in range(20, 36)],
    'clf__min_child_weight': range(1, 6),
}

In [28]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', XGBClassifier())])

In [29]:
print(pipe)

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))])


In [30]:
# Fit
grid_search = RandomizedSearchCV(pipe,params, cv=4, n_jobs=-1, verbose=1, n_iter=200)
grid_search.fit(data.data, data.target)

Fitting 4 folds for each of 200 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 19.0min


KeyboardInterrupt: 

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [25]:
# Predictions on test sample
pred = grid_search.predict(test['description'])
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

# Make Sure the Category is an Integer
submission.head()

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
subNumber = 0

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [36]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [37]:
doc = nlp("Two bananas in pyjamas")

In [38]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [39]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [40]:
X = get_word_vectors(data.data)

len(X) == len(data.data)

True

In [41]:
rfc.fit(X, data.target)

/home/cowley/venv/U4-S1-NLP/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [42]:
train[["description", "ratingCategory"]].head()

,description,ratingCategory
0,"\nSometimes, when whisky is batched, a few lef...",1
1,\nAn uncommon exclusive bottling of a 6 year o...,0
2,\nThis release is a port version of Amrut’s In...,1
3,\nThis 41 year old single cask was aged in a s...,1
4,"\nQuite herbal on the nose, with aromas of dri...",1


## Follow Along

In [44]:
nlp(train["description"][0]).vector

array([-4.67550606e-02,  2.17486411e-01, -1.39248788e-01, -8.63811150e-02,
        1.08062603e-01,  8.21812302e-02, -6.99262992e-02, -7.88967833e-02,
       -4.63048331e-02,  1.67534256e+00, -8.29739422e-02,  8.47098455e-02,
       -1.61168594e-02, -1.15673296e-01, -1.00931458e-01, -1.48674801e-01,
       -2.86327489e-02,  1.00717258e+00, -8.84611458e-02, -3.00116055e-02,
       -2.12408844e-02, -1.98691525e-03, -2.18204875e-02, -2.43686680e-02,
       -8.50067884e-02, -5.28534129e-02, -3.88755389e-02,  2.85331737e-02,
        9.71467644e-02, -1.52220830e-01, -7.67232990e-03,  3.68110649e-02,
        3.03680412e-02, -3.20375822e-02,  4.51759845e-02, -2.22320929e-02,
        2.99556088e-02, -3.53264883e-02, -8.09046626e-02,  4.02572937e-02,
       -4.52380776e-02,  4.00653407e-02,  1.01217575e-01, -1.02363355e-01,
        6.43703416e-02,  1.21800080e-01, -8.15031528e-02, -7.51190484e-02,
        4.85178418e-02,  3.61067131e-02, -5.62809184e-02,  4.18828800e-02,
       -9.56438202e-03, -

In [102]:
import re
from spacy.tokenizer import Tokenizer
from collections import Counter

tokenizer = Tokenizer(nlp.vocab)


class MyVectorizer:
    
    def __init__(self):
        self.__counter = Counter()
        
    def __clean(self, s):
        doc = tokenizer(s)
        doc_tokens = [
            re.sub(r"[^a-z0-9]", "", t.lemma_.lower()).strip() for t in doc
            if not t.is_stop and not t.is_punct and t.text.strip()
        ]
        return " ".join(doc_tokens)

    
    def fit_transform(self, docs):
        self.fit(docs)
        return self.transform(docs)
    
    
    def fit(self, docs):
        cleaned = [self.__clean(s) for s in docs]
        
        self.__counter = Counter()
        [self.__counter.update(set(s.split(" "))) for s in cleaned]
    
    
    def transform(self, docs):
        cleaned = [self.__clean(s) for s in docs]
        total = len(cleaned)
        word_to_pct = dict()
        for word, count in  self.__counter.items():
            pct = count/total
            if 0.001 <= pct and pct <= 0.95:
                word_to_pct[word] = pct
            else:
                pass

        cleaned2 = [
            " ".join(w for w in s.split(" ") if w in word_to_pct)
            for s in cleaned
        ]

        return [nlp(s).vector for s in cleaned2]

vectorizer = MyVectorizer()
X_train = pd.DataFrame(vectorizer.fit_transform(train["description"]))

Total: 4087, included: 3654


In [103]:
X_train.head(20)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.028498,0.188128,-0.138604,-0.122122,0.177986,-0.026991,-0.103738,0.041936,-0.013870,1.248903,...,-0.128297,0.021213,-0.064529,-0.045900,-0.143962,0.008417,0.034709,-0.304484,-0.020319,0.040548
1,-0.015520,0.189974,-0.016040,0.006239,0.001201,-0.064801,-0.077039,0.021943,-0.072463,1.150055,...,-0.133380,0.176411,-0.076733,-0.170855,-0.103147,0.011155,0.099502,-0.207345,0.005043,0.089354
2,0.120923,0.126532,-0.131240,-0.040462,0.074998,-0.160508,-0.061130,0.095303,0.017329,1.068527,...,-0.040477,0.021846,-0.084266,-0.166037,-0.094324,0.001587,0.107538,-0.234552,-0.162804,0.066963
3,-0.096525,0.210867,-0.070901,-0.088450,-0.008463,-0.064484,-0.179191,0.086961,0.029514,1.134071,...,-0.044764,0.089584,-0.131383,-0.172626,-0.203195,0.036219,0.135700,-0.393445,-0.038131,0.109669
4,-0.136679,0.232677,-0.210757,-0.048334,-0.094839,0.025923,-0.332620,0.095476,0.047830,0.785689,...,0.019904,0.148655,-0.159560,-0.353477,-0.343024,0.007143,0.193477,-0.583076,0.031495,0.137693
5,-0.154905,0.221724,-0.137159,-0.160858,0.104132,0.018887,-0.051697,0.015071,-0.082452,1.294517,...,-0.130995,0.140425,-0.163183,-0.140591,-0.114801,0.072113,0.051055,-0.394038,-0.032219,0.166105
6,0.088769,0.030661,-0.060032,0.026390,0.001968,-0.105039,-0.050725,-0.034064,-0.040815,1.267221,...,-0.106803,0.148391,-0.044019,-0.148248,-0.191404,-0.006327,-0.006516,-0.138992,-0.035500,0.142647
7,-0.095956,0.231727,-0.058846,-0.070600,-0.025759,-0.075900,-0.088808,0.114123,-0.018964,0.990942,...,-0.054586,0.177348,-0.138888,-0.229308,-0.272857,0.021449,0.174141,-0.327706,-0.045108,0.006042
8,-0.155937,0.158490,-0.092533,0.013157,0.024642,-0.062582,-0.132807,0.059727,0.080933,1.462363,...,-0.105423,0.204754,-0.185349,-0.105309,-0.092981,0.016343,0.135142,-0.306221,-0.055026,0.179862
9,-0.020858,0.184540,-0.097321,-0.116242,0.032797,-0.017879,-0.060531,0.059191,-0.040205,0.970501,...,-0.103237,0.167464,-0.185512,-0.222649,-0.159381,0.071477,0.134108,-0.369714,-0.035182,0.117496


In [105]:
pd.DataFrame(vectorizer.transform(train["description"])).head(20)

Total: 4087, included: 3654


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.028498,0.188128,-0.138604,-0.122122,0.177986,-0.026991,-0.103738,0.041936,-0.013870,1.248903,...,-0.128297,0.021213,-0.064529,-0.045900,-0.143962,0.008417,0.034709,-0.304484,-0.020319,0.040548
1,-0.015520,0.189974,-0.016040,0.006239,0.001201,-0.064801,-0.077039,0.021943,-0.072463,1.150055,...,-0.133380,0.176411,-0.076733,-0.170855,-0.103147,0.011155,0.099502,-0.207345,0.005043,0.089354
2,0.120923,0.126532,-0.131240,-0.040462,0.074998,-0.160508,-0.061130,0.095303,0.017329,1.068527,...,-0.040477,0.021846,-0.084266,-0.166037,-0.094324,0.001587,0.107538,-0.234552,-0.162804,0.066963
3,-0.096525,0.210867,-0.070901,-0.088450,-0.008463,-0.064484,-0.179191,0.086961,0.029514,1.134071,...,-0.044764,0.089584,-0.131383,-0.172626,-0.203195,0.036219,0.135700,-0.393445,-0.038131,0.109669
4,-0.136679,0.232677,-0.210757,-0.048334,-0.094839,0.025923,-0.332620,0.095476,0.047830,0.785689,...,0.019904,0.148655,-0.159560,-0.353477,-0.343024,0.007143,0.193477,-0.583076,0.031495,0.137693
5,-0.154905,0.221724,-0.137159,-0.160858,0.104132,0.018887,-0.051697,0.015071,-0.082452,1.294517,...,-0.130995,0.140425,-0.163183,-0.140591,-0.114801,0.072113,0.051055,-0.394038,-0.032219,0.166105
6,0.088769,0.030661,-0.060032,0.026390,0.001968,-0.105039,-0.050725,-0.034064,-0.040815,1.267221,...,-0.106803,0.148391,-0.044019,-0.148248,-0.191404,-0.006327,-0.006516,-0.138992,-0.035500,0.142647
7,-0.095956,0.231727,-0.058846,-0.070600,-0.025759,-0.075900,-0.088808,0.114123,-0.018964,0.990942,...,-0.054586,0.177348,-0.138888,-0.229308,-0.272857,0.021449,0.174141,-0.327706,-0.045108,0.006042
8,-0.155937,0.158490,-0.092533,0.013157,0.024642,-0.062582,-0.132807,0.059727,0.080933,1.462363,...,-0.105423,0.204754,-0.185349,-0.105309,-0.092981,0.016343,0.135142,-0.306221,-0.055026,0.179862
9,-0.020858,0.184540,-0.097321,-0.116242,0.032797,-0.017879,-0.060531,0.059191,-0.040205,0.970501,...,-0.103237,0.167464,-0.185512,-0.222649,-0.159381,0.071477,0.134108,-0.369714,-0.035182,0.117496


In [69]:
xgb = XGBClassifier(n_jobs=-1, learning_rate=0.22, n_estimators=120, min_child_weight=2)
xgb.fit(X_train, train["ratingCategory"])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.22,
       max_delta_step=0, max_depth=3, min_child_weight=2, missing=None,
       n_estimators=120, n_jobs=-1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [ ]:
param_dist = {
    "xgb__learning_rate": [x/100. for x in range(18, 33)],
    "xgb__n_estimators": range(80, 141, 20),
    "xgb__min_child_weight": range(0, 5),
    "xgb__max_depth": range(15, 31, 5),
}

search = RandomizedSearchCV(XGBClassifier(), param_dist, n_jobs=-1, verbose=10, cv=4, n_iter=200)
search.fit(X_train, train["ratingCategory"])
print(search.best_score_, search.best_params_)

Fitting 4 folds for each of 200 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 25

In [ ]:
X_test = vectorizer.transform(test["description"])
pred = search.predict(X_test)

In [ ]:

# Predictions on test sample
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

# Make Sure the Category is an Integer
submission.head()

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on